In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow1/free_chat/chinese_lccc/main')

In [2]:
%tensorflow_version 1.x
!pip install bert4keras

TensorFlow 1.x selected.
     |████████████████████████████████| 40kB 1.8MB/s 
  Created wheel for bert4keras: filename=bert4keras-0.8.7-cp36-none-any.whl size=39155 sha256=899282078f1c34e067a533220f51c4ff0707eaeef19fa715b12bdaa1ab236bbf
  Stored in directory: /root/.cache/pip/wheels/53/71/4c/3c37d5b70183ce174cbf51700bef11a3dcd11370ccf47052ff
Successfully built bert4keras


In [3]:
import numpy as np
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer
from bert4keras.snippets import AutoRegressiveDecoder
from bert4keras.snippets import uniout

Using TensorFlow backend.


In [4]:
config_path = '../model/GPT_LCCC-base-tf/gpt_config.json'
checkpoint_path = '../model/GPT_LCCC-base-tf/gpt_model.ckpt'
dict_path = '../model/GPT_LCCC-base-tf/vocab.txt'

tokenizer = Tokenizer(dict_path, do_lower_case=True)
speakers = [
    tokenizer.token_to_id('[speaker1]'),
    tokenizer.token_to_id('[speaker2]')
]

model = build_transformer_model(
    config_path=config_path,
    checkpoint_path=checkpoint_path,
    model='gpt_openai'
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [5]:
class ChatBot(AutoRegressiveDecoder):
    @AutoRegressiveDecoder.wraps(default_rtype='probas')
    def predict(self, inputs, output_ids, states):
        token_ids, segment_ids = inputs
        curr_segment_ids = np.zeros_like(output_ids) + token_ids[0, -1]
        token_ids = np.concatenate([token_ids, output_ids], 1)
        segment_ids = np.concatenate([segment_ids, curr_segment_ids], 1)
        return model.predict([token_ids, segment_ids])[:, -1]

    def response(self, texts, topk=5):
        token_ids = [tokenizer._token_start_id, speakers[0]]
        segment_ids = [tokenizer._token_start_id, speakers[0]]
        for i, text in enumerate(texts):
            ids = tokenizer.encode(text)[0][1:-1] + [speakers[(i + 1) % 2]]
            token_ids.extend(ids)
            segment_ids.extend([speakers[i % 2]] * len(ids))
            segment_ids[-1] = speakers[(i + 1) % 2]
        results = self.random_sample([token_ids, segment_ids], 1, topk)
        return tokenizer.decode(results[0])

In [6]:
chatbot = ChatBot(start_id=None, end_id=tokenizer._token_end_id, maxlen=32)

query_li = [
  '你好',
  '早上好',
  '晚上好',
  '再见',
  '好久不见',
  '想死你了',
  '谢谢你',
  '爱你',
  '你叫什么名字',
  '你几岁了',
  '现在几点了',
  '今天天气怎么样',
  '我们现在在哪里',
  '你能给我讲个笑话吗',
  '你是男孩还是女孩呀',
  '你会几种语言呀',
  '你能陪我玩吗',
  '说话可以大声一点吗',
  '天气真好',
  '天气太糟糕了',
  '下雨了',
  '雨好大',
  '我讨厌艳阳天',
  '好晒啊',
  '今天好冷',
  '今天好热',
  '风好大',
  '雾太大了看不清路',
  '打雷了好可怕',
  '下雪了诶',
  '好烦啊',
  '好开心',
  '太激动了',
  '我好难过',
  '我想哭',
  '太好笑了',
  '我好伤心',
  '心好痛',
  '好累啊',
  '我好疲惫',
  '我爱你',
  '我讨厌你',
  '你真是太棒啦',
  '你好厉害啊',
  '吓死我了',
  '我想回家',
  '我想爸妈了',
  '不知道小孩在家有没有听话',
  '想回家撸猫',
]
for q in query_li:
  print('Q:', q)
  print('A:', chatbot.response([q]))
  print()

Q: 你好

A: 你好

Q: 早上好
A: 早上好，好久不见

Q: 晚上好
A: 你好

Q: 再见
A: 不见

Q: 好久不见
A: 是呀，好久不见。

Q: 想死你了
A: 我也想你，

Q: 谢谢你
A: 不用谢

Q: 爱你
A: 你是爱我

Q: 你叫什么名字
A: 我也想知道

Q: 你几岁了
A: 你猜

Q: 现在几点了
A: 你说呢？

Q: 今天天气怎么样
A: 今天很冷，但是我还是觉得很冷

Q: 我们现在在哪里
A: 我在这里等你呢

Q: 你能给我讲个笑话吗
A: 讲个笑话

Q: 你是男孩还是女孩呀
A: 你猜

Q: 你会几种语言呀
A: 不会

Q: 你能陪我玩吗
A: 可以

Q: 说话可以大声一点吗
A: 可以的

Q: 天气真好
A: 是啊！

Q: 天气太糟糕了
A: 是啊

Q: 下雨了
A: 你们那边也下雨了

Q: 雨好大
A: 是啊

Q: 我讨厌艳阳天
A: 我也是。

Q: 好晒啊
A: 是啊

Q: 今天好冷
A: 你们都不来

Q: 今天好热
A: 热死我了

Q: 风好大
A: 风太大了。。。

Q: 雾太大了看不清路
A: 哈哈，你这是在逗我笑

Q: 打雷了好可怕
A: 我们这边也是，我们这边下雨了，我们这边也没有下雨。

Q: 下雪了诶
A: 是滴！

Q: 好烦啊
A: 我也是

Q: 好开心
A: 你是不是想我了

Q: 太激动了
A: 我也激动

Q: 我好难过
A: 我也是，我们都是

Q: 我想哭
A: 抱紧你

Q: 太好笑了
A: 你也笑了？

Q: 我好伤心
A: 你伤心什么

Q: 心好痛
A: 你心痛什么

Q: 好累啊
A: 你不会还是一个人在寝室吧

Q: 我好疲惫
A: 我也差不多。。

Q: 我爱你
A: 我也是

Q: 我讨厌你
A: 你不是一直很喜欢我

Q: 你真是太棒啦
A: 你最棒

Q: 你好厉害啊
A: 是不是

Q: 吓死我了
A: 我都不敢出门了

Q: 我想回家
A: 我也想！！

Q: 我想爸妈了
A: 哈哈

Q: 不知道小孩在家有没有听话
A: 没有

Q: 想回家撸猫
A: 你也想啊？

